In [1]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


branca-0.3.1         | 25 KB     | #################################

## Introduction

The amount of crime in India is rising everyday. Crimes are categorized into multiple different groups. Our goal in this project is to figure out which states in the country has the most number of a certain crime. We finally conclude with which state has the most overall crimes.

## Data

The dataset is take from Kaggle. It contains information about the trials of violent crimes by courts and will eventually help determine the kind of results we watn. Foursquare data is leveraged in order to be able to plot our final results on a map.

In [7]:
import pandas as pd

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_55b2bb94579d4d9ba04aaf80a8aad6ca = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WhvSgNGjSkNqmccQXa2b1IlJ7LFPf64FsfiV_oD6jSMq',
    ibm_auth_endpoint="https://iam.au-syd.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_55b2bb94579d4d9ba04aaf80a8aad6ca.get_object(Bucket='couseracapstone-donotdelete-pr-3vhu2vlrhbzpzg',Key='28_Trial_of_violent_crimes_by_courts.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


In [10]:
crime=pd.read_csv('https://www.kaggle.com/rajanand/crime-in-india/download/knRd4Yf8iPdFLZS16BOn%2Fversions%2FQItWwVr2cdwEt45EuZLu%2Ffiles%2F28_Trial_of_violent_crimes_by_courts.csv?datasetVersionNumber=1')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 2
